In [75]:
import os
import time
import pandas as pd
import urllib
from urllib.request import urlopen, urlretrieve

from bs4 import BeautifulSoup
from tqdm import tqdm
from shutil import rmtree

In [4]:
csv = pd.read_csv("./kstr_data.csv")
case_links = csv["link"]

In [5]:
case_links[0].split("number=")[-1]

'324'

In [6]:
base_link = "https://kstr.radiology.or.kr/weekly/files"
images = []
pbar = tqdm(case_links)
index = 1
for link in pbar:
    pbar.set_description(f"Crawling case {index:4}\t\t")

    response = urlopen(link)
    soup = BeautifulSoup(response, 'html.parser')

    image_count = len(soup.find("ul", {"class" : "thumbList"}).find_all("li"))
    
    case_num = link.split("number=")[-1]
    image_links = [f"{base_link}/case-{case_num}-{i+1}.jpg" for i in range(image_count)]

    images.append(image_links)
    index += 1
    #pbar.set_description(f"Now waiting before case {index:4}\t")
    #time.sleep(random.randint(5, 15)

Crawling case    1		:   0%|          | 0/1342 [00:00<?, ?it/s]

Crawling case 1342		: 100%|██████████| 1342/1342 [11:48<00:00,  1.89it/s] 


In [168]:
errors = []

for i, links in enumerate(tqdm(images)):
    if not (i+1 in errors): continue
    dirname = f"./images/case_{(i+1):04}"
    if not os.path.isdir(dirname): os.mkdir(dirname)
    extensions = ["jpg", "JPG", 'gif', "GIF", "png", "PNG", "jpeg", "JPEG"]
    
    for j, l in enumerate(links, start=1):
        success = False
        for ex in extensions:
            try:
                urlretrieve(l.replace('jpg',ex), os.path.join(dirname, f"image_{j}.{ex}"))
                success = True
                break
            except:
                continue
            
        if not success:
            errors.append(i+1)
            rmtree(dirname)
            print('asd')
            break


100%|██████████| 1342/1342 [00:00<00:00, 5100.55it/s]


In [165]:
error_cases=[
    {
        "id" : i,
        "link" : f"https://kstr.radiology.or.kr/weekly/archive/view.php?number={images[i-1][0].split('case-')[-1].split('-')[0]}"
    }
    for i in errors
]

In [167]:
index = 636
# for index in errors_take3:
#     index =
#     images[index] = [link.replace(".JPG",".jpg") for link in images[index]]
# images[index][4] = images[index][4].replace(".GIF",".JPG")
images[index]

['https://kstr.radiology.or.kr/weekly/files/case-735-1.jpg',
 'https://kstr.radiology.or.kr/weekly/files/case-735-2.jpg',
 'https://kstr.radiology.or.kr/weekly/files/case-735-3.jpg']

In [142]:
images[635][2] = images[635][2].replace(".jpg", ".gif")

In [187]:
from PIL import Image
for case_dir in tqdm(os.listdir("./kstr_images")):
    if len(os.listdir(f"./kstr_images/{case_dir}")) == 0: print(case_dir)
            
            

 46%|████▌     | 616/1342 [00:00<00:01, 673.62it/s]

case_0485


100%|██████████| 1342/1342 [00:01<00:00, 673.76it/s]


In [188]:
len(sum(images, []))

6287

In [15]:
pd.DataFrame(images).to_csv("kstr_images.csv", index=False, header=False)